In [1]:
import sys

sys.path.append("..")

In [2]:
from config import config
from config import config_indicators

from preprocessor.findata_preprocessor import FinancialDataPreprocessor
from visualiser.findata_visualiser import FinancialDataVisualiser
from environments.env_finrl import FinRLTradingEnv

In [3]:
finpreprocessor = FinancialDataPreprocessor(config.START_DATE, config.END_DATE)
train_data, trade_data = finpreprocessor.load_train_test_data(
    "../" + config.DATA_DIR, config.TEST_NAME
)

In [4]:
finvisualiser = FinancialDataVisualiser()
finvisualiser.plot_train_test_close_prices(
    train_data, trade_data, "../" + config.PLOT_DIR, config.TEST_NAME
)

/Users/ingridperez/Documents/GitHub Repositiories/xdl-portfolio/examples/../visualiser/findata_visualiser.py:171: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
environment = FinRLTradingEnv(
    train_data, trade_data, list(config_indicators.TECHNICAL_INDICATORS.keys())
)

Environment sucessfully created with 
	Stock dimension: 5 
	State space: 91


In [6]:
env_train = environment.get_train_env()

In [7]:
gym_env, (env_trade, obs_trade) = environment.get_trade_env()